# LaTeX Automation for NYCHA Waste Individual Action Plans

In [163]:
import sys
import os
import datetime
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from PIL import Image
import docx2txt
import re

In [2]:
#Set Global Vars and Options
os.chdir('/Users/kyleslugg/Documents/NYCHA/Production')
cons_tds = '073'
pd.set_option('display.max_columns', None)

## Parse and Process Text Blocks

TO COME:
- Waste Services and Assets
- Waste Distribution (top text -- bottom accounted for below)
- What Is an IAP?

In [3]:
#Character Substitutions for LaTeX -- set and define "clean" method
def clean_text(text):
    substitutions = {'“':"``",
                '”': "''",
                '’':"'",
                ' ':' ',
                '–':'--',
                ' ':' ',
                '\xa0':' '}
    
    for key, value in substitutions.items():
        text = text.replace(key, value)
        
    return text

#### Overview Text

In [4]:
def make_overview_text(cons_tds):
    header = re.compile(r'((\w*\s)*(Overview)):?')
    
    overview_text = docx2txt.process(f'TEXT/{cons_tds}_Overview.docx')
    try:
        overview_text = overview_text.replace(header.findall(overview_text)[0]+'\n','')
    except:
        overview_text = overview_text.replace(header.findall(overview_text)[0][0]+'\n','')
    overview_text = clean_text(overview_text)
    
    with open(f'TEXT/overview_text/{cons_tds}_overview.tex', 'w') as file_handle:
        file_handle.write(overview_text)
    
    pass

In [5]:
make_overview_text(cons_tds)

#### Analysis Text

In [6]:
def make_analysis_text(cons_tds):
    analysis_text = docx2txt.process(f'TEXT/{cons_tds}_Analysis.docx')

    header = re.compile(r'([\w\s]*:)')

    section_headings = {'Inspection and Collection Requirement':['Inspection and Collection Requirement',
                                                                'Inspection and Collection Requirements',
                                                                'Collection and Inspection Requirement',
                                                                'Collection and Inspection Requirements'],
                        
                        'Removal or Storage Requirement':['Removal or Storage Requirement',
                                                          'Removal or Storage Requirements',
                                                         'Removal and Storage Requirement',
                                                         'Removal and Storage Requirements',
                                                         'Storage or Removal Requirement',
                                                         'Storage and Removal Requirement',
                                                         'Storage and Removal Requirements']}
    
    for heading, variants in section_headings.items():
        for variant in variants:
            analysis_text = analysis_text.replace(variant, r'\emph{%s}' % heading)

    analysis_text = analysis_text.replace(header.findall(analysis_text)[0]+'\n','')

    latex_block = clean_text(analysis_text)

    with open(f'TEXT/analysis_text/{cons_tds}_analysis.tex', 'w') as file_handle:
        file_handle.write(latex_block)
        
    pass

In [7]:
make_analysis_text(cons_tds)

## Select and Prepare Maps

In [8]:
# Set asset map path
asset_map_path = f"MAPS/asset_maps/{cons_tds}_asset_map.png"

In [9]:
# Split context map into two pages
def process_context_map(cons_tds):
    image = Image.open(f'REPORT_TEMPLATE/{cons_tds}_context_map.png')
    width, height = image.size

    bb1 = (0,0,width/2,height)
    bb2 = (width/2, 0, width, height)

    img_1 = image.crop(bb1)
    img_2 = image.crop(bb2)

    img_1.save(f'MAPS/context_maps/{cons_tds}_context_1.png', format="PNG")
    img_2.save(f'MAPS/context_maps/{cons_tds}_context_2.png', format="PNG")
    
    pass

## Produce Tables

TO COME:
- Waste Services and Assets
- Assets

In [115]:
#Load Data
overview_data = pd.read_csv('DATA/overview_table_data.csv')
overview_data['CONS_TDS'] = overview_data['CONS_TDS'].apply(lambda x: str(x).zfill(3))
overview_data['TDS'] = overview_data['TDS'].apply(lambda x: str(x).zfill(3))

#### Make Overview Table

In [116]:
def make_overview_table(cons_tds, overview_data=overview_data):
    
    cons_data = overview_data.loc[overview_data['CONS_TDS']== cons_tds]
    
    overview_table = ''

    overview_frame = r'''
    \begin{tabular}{l|c|c|c|c|}
    \cline{2-5}
                                                                           & \cellcolor{ccteal}{\color[HTML]{FFFFFF} TDS \#} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Total Households} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Official Population} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Average Family Size} \\ \hline

    '''

    development_template = r'''\multicolumn{1}{|l|}{\cellcolor{ccteallight}%s}        & %s                                                   & %s                                                           & %s                                                                & %s                                                                \\ \hline'''


    overview_table += overview_frame

    for row in cons_data.itertuples():
        dev_name = row.DEV_NAME.title()
        dev_tds = row.TDS
        total_hhs = row.TOTAL_HH
        official_population = row.TOTAL_POP
        avg_family_size = row.AVG_FAMILY_SIZE

        overview_table += development_template % (dev_name, dev_tds, total_hhs, official_population, avg_family_size)

    overview_table += r'''
    \end{tabular}
    '''
    
    with open(f'TABLES/overview_table/{cons_tds}_overview_table.tex', 'w') as file_handle:
        file_handle.write(overview_table)
    

#### Typology Table

In [162]:
typ_1 = pd.read_csv('DATA/typologies_1.csv')
typ_2 = pd.read_csv('DATA/typologies_2.csv')

typ_1.columns = ['CONS_NAME', 'DEV_NAME', 'TDS', 'TYPOLOGY']
typ_2.columns = ['CONS_NAME', 'CONS_TDS', 'DEV_NAME', 'TDS', 'METHOD', 
                 'CONSTRUCTION_DATE', 'BLDG_AGE', 'STORIES', 'BLDG_COVERAGE_SQFT', 'OPEN_SPACE_RATIO', 'SCATTERED_SITE_FLAG']

typ_2['CONSTRUCTION_DATE'] = typ_2['CONSTRUCTION_DATE'].apply(lambda x: pd.to_datetime(x))

In [165]:
typ_2['SCATTERED_SITE_FLAG'] = typ_2['SCATTERED_SITE_FLAG'].apply(lambda x: x == 'YES')
typ_2.loc[typ_2['SCATTERED_SITE_FLAG']=='YES','SCATTERED_SITE_FLAG'] = 1

typology = typ_1.merge(typ_2[['CONS_TDS', 'TDS', 'METHOD',
                             'CONSTRUCTION_DATE', 'BLDG_AGE', 
                             'STORIES', 'BLDG_COVERAGE_SQFT', 
                             'OPEN_SPACE_RATIO', 'SCATTERED_SITE_FLAG']], how='left', on='TDS')

typology['PREWAR'] = typology['CONSTRUCTION_DATE'].apply(lambda x: x < datetime.date(1945,1,1))

typology.head()

,CONS_NAME,DEV_NAME,TDS,TYPOLOGY,CONS_TDS,METHOD,CONSTRUCTION_DATE,BLDG_AGE,STORIES,BLDG_COVERAGE_SQFT,OPEN_SPACE_RATIO,SCATTERED_SITE_FLAG,PREWAR
0,1010 EAST 178TH STREET,1010 EAST 178TH STREET,180,1 - High-rise in the park,180.0,CONVENTIONAL,1971-03-31,49.0,21,"14,961",0.83,False,False
1,1010 EAST 178TH STREET,EAST 180TH STREET-MONTEREY AVENUE,208,5 - Context Mid-rises,180.0,CONVENTIONAL,1973-09-30,47.0,10,"30,800",0.53,False,False
2,1010 EAST 178TH STREET,TWIN PARKS EAST (SITE 9),287,2 - Mid-rise in the park,180.0,CONVENTIONAL,1982-04-30,38.0,14,"11,388",0.82,False,False
3,Adams,ADAMS,118,1 - High-rise in the park,118.0,CONVENTIONAL,2064-08-31,56.0,15-21,"56,283",0.86,False,False
4,Boston Secor,BOSTON SECOR,138,1 - High-rise in the park,138.0,CONVENTIONAL,2069-04-30,51.0,13-14-17-18,"36,181",0.92,False,False


In [ ]:

header_two = r"\begin{tabular}{m{1.25in} m{2in} m{.1in} m{1.25in} m{2in}}"
header_three = r"\begin{tabular}{m{1.25in} m{2in} m{.1in} m{1.25in} m{2in} m{1.25in} m{2in}}"
header_three = r"\begin{tabular}{m{1.25in} m{2in} m{.1in} m{1.25in} m{2in} m{1.25in} m{2in} m{1.25in} m{2in}}"
r'''\begin{tabular}{m{1.25in} m{2in} m{.1in} m{1.25in} m{2in}}
\sf\bf{Sumner Houses and 303 Vernon Avenue} & \includegraphics[height=2in]{towers_in_park} & & \sf\bf{Bedford-Stuyvesant Rehab} & \includegraphics[height=2in]{prewar.png}
\end{tabular}'''

#### Waste Calculator

In [117]:
def add_waste_cols(overview_data=overview_data):
    conversion_factors = {'units_to_tons_day': 0.0025,
                         'cy_per_ton': {'trash': 21.05,
                                        'trash_actual': 0,
                                       'MGP': 18.02,
                                       'cardboard': 26.67,
                                       'paper': 6.19,
                                       'organics': 4.32,
                                       'ewaste': 5.65,
                                       'textiles': 13.33},
                         'gallons_per_cy': 201.974,
                         'gallons_per_64gal': 64,
                         'gallons_per_40lb_bag': 44,
                         'cy_per_44gal_bag':0.174,
                         'cy_per_cardboard_bale':0.193}

    waste_percentages = {'trash': .26,
                         'trash_actual':.894,
                        'MGP': .19,
                        'cardboard': .07,
                        'paper': .07,
                        'organics':.32,
                        'ewaste': .01,
                        'textiles': .08}

    capture_rates = {'trash_primary': .75,
                    'trash_secondary': .25,
                    'mgp': .30,
                    'cardboard': .50,
                    'paper': .20}

    overview_data['WASTE_TONS_DAY'] = overview_data['CURRENT_APTS'].apply(lambda x: x * conversion_factors['units_to_tons_day'])

    for key, value in waste_percentages.items():
        overview_data[f'{key.upper()}_CY'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value * conversion_factors['cy_per_ton'][key])
        overview_data[f'{key.upper()}_TONS'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value)
    
    overview_data['TRASH_ACTUAL_CY'] = (overview_data['TRASH_CY']+
                                           overview_data['MGP_CY']+
                                           overview_data['CARDBOARD_CY']+
                                           overview_data['PAPER_CY']+
                                           overview_data['ORGANICS_CY']+
                                           overview_data['EWASTE_CY']+
                                           overview_data['TEXTILES_CY'])-(overview_data['MGP_CY']*capture_rates['mgp']+
                                                                         overview_data['CARDBOARD_CY']*capture_rates['cardboard']+
                                                                         overview_data['PAPER_CY']*capture_rates['paper'])

    overview_data['TRASH_CHUTE_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_SAUSAGE'] = ((overview_data['TRASH_CHUTE_CY'])/conversion_factors['cy_per_ton']['trash'])*(2000/40)
    overview_data['TRASH_DROP_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_BINS'] = overview_data['TRASH_DROP_CY']*conversion_factors['gallons_per_cy']/64
    overview_data['CAPTURED_MGP_TONS_WEEK'] = overview_data['MGP_TONS']*capture_rates['mgp']*7
    overview_data['CAPTURED_CARDBOARD_TONS_WEEK'] = overview_data['CARDBOARD_TONS']*capture_rates['cardboard']*7
    overview_data['CAPTURED_PAPER_TONS_WEEK'] = overview_data['PAPER_TONS']*capture_rates['paper']*7
    overview_data['MGP_BAGS_WEEK'] = overview_data['MGP_CY']*capture_rates['mgp']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['PAPER_BAGS_WEEK'] = overview_data['PAPER_CY']*capture_rates['paper']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['CARDBOARD_BALES_WEEK'] = overview_data['CARDBOARD_CY']*capture_rates['cardboard']*7/conversion_factors['cy_per_cardboard_bale']
    
    return overview_data

In [122]:
overview_data = add_waste_cols(overview_data)

In [125]:
def make_waste_distribution_table(cons_tds, overview_data=overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]
    
    if num_devs > 1:
        num_cols = num_devs+1
    else:
        num_cols = num_devs

    dev_col_format = r'X|'

    opening = r'''
    \begin{tabularx}{\textwidth}{V{1.5in}|%s}
    \cline{2-%s}
    ''' % (dev_col_format*num_cols, (num_cols+1))

    top_row = r'''
                                                                   '''+(r"& \multicolumn{1}{l|}{\cellcolor{ccorange}%s}"*(num_cols))+r"\tnhl"+'\n'

    standard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    captured_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Captured / Week (tons)\tnote{4}}                        "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    chute_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Trash Chutes\tnote{2}}                 "+(r"& %s tons or (%s) 40 lbs. sausage bags      "*num_cols)+r"\tnhl"+'\n'

    dropsite_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Drop Sites\tnote{3}}                 "+(r"& %s tons or (%s) 64-gallon bins      "*num_cols)+r"\tnhl"+'\n'

    OET_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s / Day (CY)\tnote{5}}              "+(r"& %s                                    "*num_cols)+r"\tnhl"+'\n'
    
    recycling_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) 44-gallon bags                                   ")*num_cols+r"\tnhl"+'\n'
    
    cardboard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) bales                                   ")*num_cols+r"\tnhl"+'\n'
    
    def make_waste_distribution_table_block(cons_data, num_cols):
        if num_cols != 1:
            cons_data.loc['Total']= cons_data.sum(numeric_only=True, axis=0)
            cons_data.loc['Total','DEV_NAME'] = 'Total'

        def make_trash_text(row, text_var, cy_col, other_col):
            text_var.append(round(row[cy_col],2))
            text_var.append(round(row[other_col], 2))
            pass

        latex_block = opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        latex_block += standard_row % tuple([r"Waste Generated / Day (Tons)\tnote{1}"]+[round(item, 2) for item in cons_data['WASTE_TONS_DAY'].tolist()])
        latex_block += standard_row % tuple([r"Trash / Day (tons)\tnote{2}"]+cons_data['TRASH_ACTUAL_TONS'].apply(lambda x: str(round(x,2))).tolist())

        trash_chute_text = []
        dropsite_text = []

        cons_data.apply(lambda row: make_trash_text(row, trash_chute_text, 'TRASH_CHUTE_TONS', 'TRASH_CHUTE_SAUSAGE'), axis=1)
        cons_data.apply(lambda row: make_trash_text(row, dropsite_text, 'TRASH_DROP_TONS', 'TRASH_DROP_BINS'), axis=1)

        latex_block += chute_row % tuple(trash_chute_text)
        latex_block += dropsite_row % tuple(dropsite_text)
        
        latex_block += r"\end{tabularx}\bigskip"
        
        latex_block += opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        
        mgp_text = []
        cardboard_text= []
        paper_text = []
        
        cons_data.apply(lambda row: make_trash_text(row, mgp_text, 'CAPTURED_MGP_TONS_WEEK', 'MGP_BAGS_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, cardboard_text, 'CAPTURED_CARDBOARD_TONS_WEEK', 'CARDBOARD_BALES_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, paper_text, 'CAPTURED_PAPER_TONS_WEEK', 'PAPER_BAGS_WEEK'), axis=1)


        
        latex_block += recycling_row % tuple([r"Metal, Glass, Plastic Captured / Week (tons)"]+mgp_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_MGP_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += cardboard_row % tuple([r"Cardboard Captured / Week (tons)"]+cardboard_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_CARDBOARD_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += recycling_row % tuple([r"Paper Captured / Week (tons)"]+paper_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_PAPER_CY'].apply(lambda x: str(round(x,2))).tolist())

        #latex_block += OET_row % tuple(['Organics']+cons_data['ORGANICS_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['E-Waste']+cons_data['EWASTE_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['Textiles']+cons_data['TEXTILES_CY'].apply(lambda x: str(round(x,2))).tolist())

        latex_block += r"\end{tabularx}"

        return latex_block
    
    
    latex_block = make_waste_distribution_table_block(cons_data, num_cols)
    
    with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table.tex', 'w') as file_handle:
        file_handle.write(latex_block)
    
    
    text_block = r''''''
    
    text_line_multi = r"\bf{%s}: This development has %s apartment units and %s stairhalls.\\"

    text_line_singular = r"\bf{%s}: This development has %s apartment units and one stairhall.\\"
    
    for row in cons_data.itertuples():
    
        if int(row.STAIRHALLS) == 1:
            text_block += text_line_singular % (row.DEV_NAME.title(), int(row.CURRENT_APTS))
        else:
            text_block += text_line_multi % (row.DEV_NAME.title(), int(row.CURRENT_APTS), int(row.STAIRHALLS))

    
    with open(f'TEXT/waste_distribution_bottom/{cons_tds}_wd_bottom.tex', 'w') as file_handle:
        file_handle.write(text_block)

    

In [126]:
make_waste_distribution_table('073')

#### Make Capital Improvements Table

In [14]:
asset_data = {'fwd': ['In-Sink Food Grinders', pd.read_csv('DATA/capital_fwd.csv')],
              'ehd': ['Enlarged Hopper Doors', pd.read_csv('DATA/capital_ehd.csv')],
              'int_compactor':['Interior Compactor Replacement', pd.read_csv('DATA/capital_intcom.csv')],
              'wasteyard':['Waste Yard Redesign', pd.read_csv('DATA/capital_wasteyard.csv')]}

for value in asset_data.values():
    value[1].columns = [item.strip() for item in value[1].columns]

asset_data['wasteyard'][1]['ESTIMATE'] = asset_data['wasteyard'][1]['TOT_EST']
asset_data['wasteyard'][1]['COST'] = np.nan

In [15]:
def make_capital_table(cons_tds, overview_data=overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]

    dev_col_format = r'X|'
    header = r'''
    \begin{tabularx}{\textwidth}{r|%s}
    \cline{2-%s}
    ''' % ((dev_col_format*num_devs), num_devs)

    top_row = r"\multicolumn{1}{l|}{}                                                        "+r"& \cellcolor{ccorange}{\color[HTML]{FFFFFF}%s} "*num_devs+r"\\ \hline"+"\n"

    project_block = r"\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}%s}          "+(r"&                                                                  "*num_devs)+r"\\"+r'''
    \multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                '''+(r"& %s                                                         "*num_devs)+r'''\\
    \multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{%s}}                  '''+("& %s                                                     "*num_devs)+r"\\ \hline"+"\n"
    
    def make_capital_table_block(cons_data):
        devs = cons_data['DEV_NAME'].apply(lambda x: str(x).upper()).tolist()
        devs_title = cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist()
        latex_block = ''
        latex_block += header
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())

        for asset in asset_data.keys():
            asset_df = asset_data[asset][1]
            #print(asset_data[asset][0])
            #print(devs)
            #print(asset_df['DEVELOPMENT'].tolist())
            if any((dev in asset_df['DEVELOPMENT'].tolist()) for dev in devs):
                status_list = []
                cost_title = []
                cost_list = []

                for dev in devs:
                    if dev in asset_df['DEVELOPMENT'].tolist():
                        #print(dev)
                        if pd.isna(asset_df.loc[asset_df['DEVELOPMENT']== dev,'STATUS'].iloc[0]):
                            status_list.append('Estimate')
                        else:
                            status_list.append(str(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'STATUS'].iloc[0]).title())

                        #print(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'STATUS'])
                        #print(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'COST'])

                        if (str(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'COST'].iloc[0]).strip() == '$-') or (pd.isna(asset_df.loc[asset_df['DEVELOPMENT'] == dev, 'COST'].iloc[0])):
                            if (str(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'ESTIMATE'].iloc[0]).strip() == '$-' or pd.isna(asset_df.loc[asset_df['DEVELOPMENT']==dev, 'ESTIMATE'].iloc[0])):
                                cost_list.append(' ')
                            else:
                                cost_list.append('\\'+str(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'ESTIMATE'].iloc[0])+r" (est.)")
                        else:
                            if (str(asset_df.loc[asset_df['DEVELOPMENT']==dev, 'COST'].iloc[0]).strip()== '$-' or pd.isna(asset_df.loc[asset_df['DEVELOPMENT']==dev, 'COST'].iloc[0])):
                                cost_list.append(' ')                    
                            else: 
                                cost_list.append('\\'+asset_df.loc[asset_df['DEVELOPMENT']==dev, 'COST'].iloc[0])

                    else:
                        status_list.append('N/A')
                        cost_list.append(' ')

                if all((item == ' ') for item in cost_list):
                    cost_title.append(' ')
                else:
                    cost_title.append('Cost')

                asset_block = project_block % tuple([asset_data[asset][0]]+status_list+cost_title+cost_list)

                latex_block += asset_block

        latex_block += r"\end{tabularx}"

        return latex_block
    
    capital_block = make_capital_table_block(cons_data)
    
    with open(f"TABLES/capital_projects_table/{cons_tds}_capital_projects.tex", 'w') as file_handle:
        file_handle.write(make_capital_table_block(cons_data))
    

In [16]:
make_capital_table('073')

#### Make Staff Table

In [74]:
# Read consolidation name data
cons_ids = pd.read_csv('DATA/CONS_NAME_TDS.csv')
cons_ids['CONS_TDS'] = cons_ids['CONS_TDS'].apply(lambda x: str(x).zfill(3))

#Read budgeted staff and formula allocation
dev_staff = pd.read_csv('DATA/staff_for_table.csv')
dev_staff.dropna(inplace=True)
dev_staff['Consolidation'] = dev_staff['Consolidation'].apply(lambda x: str(x).upper())
#Note: Staff list missing for Armstrong, Ft. Washington, and Williams Plaza, as well as scatter-site third-party-managed consolidations
dev_staff = dev_staff.merge(cons_ids, left_on='Consolidation', right_on='CONS_NAME', how='inner', indicator=False)

#Read budgeted staff and actuals
actuals_data = pd.read_csv('DATA/Staffing_Analysis/DEVHC.csv')
actuals_data.fillna(0, inplace=True)
actuals_data = actuals_data[actuals_data['RC Name'].apply(lambda x: "total" not in str(x).lower()) & actuals_data['Department'].apply(lambda x: "total" not in str(x).lower())]

def convert_neg(x):
    try:
        return int(x)
    except:
        return int('-'+str(x).replace('(','').replace(')',''))
    
actuals_data['VARIANCE'] = actuals_data['Unnamed: 5'].apply(lambda x: convert_neg(x))
actuals_data['ACT'] = actuals_data['13']

actuals_data = actuals_data.merge(actuals_keys, how='left', left_on='CST_NAME', right_on='TITLE_NAME')
for column in ['Current Modified', 'ACT', 'VARIANCE']:
    actuals_data[column] = actuals_data[column].astype(int)

In [108]:
# Read column keys and table keys
table_frame = pd.read_csv('DATA/Table_Keys.csv')
actuals_keys = pd.read_csv('DATA/Staffing_Analysis/DEVHC_CODES.csv')

In [111]:
def make_staff_table(cons_tds):
    cons_id = cons_ids['CONS_NAME'].loc[cons_ids['CONS_TDS'] == cons_tds].iloc[0]
    
    #Fetching staff data for consolidation
    cons_data = dev_staff.loc[dev_staff['Consolidation'] == cons_id]
    
    # Isolate and process actuals data for consolidation
    try:
        cons_actuals = actuals_data[actuals_data['RC Name'].apply(lambda x: str(x).lower() == cons_id.lower())]
    except:
        print(f'{cons_id} not found in actuals.')
        return np.NaN
    
    cons_actuals = cons_actuals[['RC Name', 'Current Modified', 'ACT', 
                                 'CODE_KEY', 'CODE_NAME']].groupby(by='CODE_KEY', as_index=False).agg({'RC Name': 'first',
                                                                                                     'Current Modified':sum,
                                                                                                     'ACT':sum,
                                                                                                     'CODE_NAME':'first'})
    cons_actuals
    cons_actuals.loc['Total']= cons_actuals.sum(numeric_only=True, axis=0)
    cons_actuals.loc['Total','CODE_KEY'] = 11
    cons_actuals.loc['Total','CODE_NAME'] = 'TOT'
    
    for row in cons_actuals.itertuples():
        cons_data[f'{row.CODE_NAME}_ACT'] = row.ACT

    #Setting up table and transposing data
    cons_table_frame = table_frame
    cons_table_frame['Formula'] = cons_table_frame['FORMULA_KEY'].iloc[:-1].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Budgeted'] = cons_table_frame['BUDG_KEY'].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Actual'] = cons_table_frame['ACTUALS_KEY'].iloc[:-2].apply(lambda key: cons_data[key].iloc[0])

    
    #Simplifying table
    cons_table = cons_table_frame[['CHART_LINE', 'Formula', 'Budgeted', 'Actual']]
    print(cons_table)
    
    #Defining LaTeX table format
    
    def make_staff_table_block(staff_data):
    
        table_template = r'''
        \begin{tabular}{l|c|c|c|}
        \cline{2-4}
                                                                                     & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Formula Allocation} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Budgeted} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Actual} \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Employees}                      & %s                                                      & %s                                                                & %s                                                        \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Property Manager}               & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Asst. Property Manager}         & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Secretaries}                    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Housing Assistants}             & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Superintendent}                 & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Assistant Superintendent}       & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Caretakers (SOC)} & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Grounds (SOG)}    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Maintenance Workers}            & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers X}                   & %s                                                      & %s                                                                &                                                       \\ \hline  \cline{1-3}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers J\tnote{1}}                   &                                                       & %s                                                                &                                                         \\ \cline{1-1} \cline{3-3}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers G}                   & \multirow{-2}{*}{%s}                                                      & %s                                     & \multirow{-3}{*}{%s}                           \\ \hline
        \end{tabular}
        
        '''

        values = []

        def extract_data_through_mw(row):
            [values.append(item) for item in [str(int(row['Formula'])), 
                                              str(int(row['Budgeted'])), 
                                              str(int(row['Actual']))]]
            pass

        #Processing through Maintenance Worker
        staff_data.iloc[0:-3].apply(lambda row: extract_data_through_mw(row), axis=1)

        #Processing Caretakers
        values.append(str(int(staff_data.iloc[-3, 1])))
        values.append(str(int(staff_data.iloc[-3, 2])))
        values.append(str(int(staff_data.iloc[-2, 2])))
        values.append(str(int(staff_data.iloc[-2, 1])))
        values.append(str(int(staff_data.iloc[-1, 2])))
        values.append(str(int(staff_data.iloc[-3, 3])))

        return table_template % tuple(values)
    
    #Make and export LaTeX code
    with open(f'TABLES/staff_table/{cons_tds}_staff_table.tex', 'w') as file_handle:
        file_handle.write(make_staff_table_block(cons_table))